In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newscompany/tata_aspect.csv
/kaggle/input/newscompany/apollo_aspect.csv
/kaggle/input/newscompany/reliance_aspect.csv


In [2]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import transformers
from transformers import AutoModel , AutoTokenizer,AutoConfig
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from tqdm import tqdm
from torch.optim import AdamW

In [4]:
df=pd.read_csv("/kaggle/input/newscompany/reliance_aspect.csv")

In [5]:
df.head()

,date,news,Predicted Sentiment,Sentiment Probability,aspect_labelling
0,2017-12-18,"So far, Reliance Industries has Hotstar and Ro...",neutral,0.828784,"[[('stake', 'stock_market', 'neutral', 0.92604..."
1,2017-12-23,The rally in the countrys most valuable compan...,positive,0.914790,"[[('Reliance', 'companies', 'positive', 0.9455..."
2,2017-12-24,Mukesh Ambani sets out five aims for Reliance ...,neutral,0.826013,"[[('Reliance', 'companies', 'neutral', 0.58129..."
3,2017-12-25,Mukesh Ambani set five aims for the company an...,neutral,0.760601,"[[('mukesh ambani', 'investor_sentiment', 'neu..."
4,2018-01-02,The latest deal between the estranged Ambani b...,positive,0.548908,"[[('Reliance', 'companies', 'neutral', 0.62014..."


In [ ]:
   def extract_embeddings(self,features,inputs):
        last_hidden_state=features[0]
        #print(features.last_hidden_state.shape)
        #print(features.keys())
        #print(len(features['hidden_states']))
        #print(features['last_hidden_state']==features["hidden_states"][0])
        #print(last_hidden_state[:,512:])
        attention_mask=inputs["attention_mask"]
        embeddings=self.pool(last_hidden_state,attention_mask)
        
        return embeddings
        
    def forward(self,inputs):
        features=self.model(**inputs)
        out=self.extract_embeddings(features,inputs)
        f1=self.fc1(out)
        f1=self.relu(f1)
        f1=self.layer_norm(f1)
        f=self.fc2(f1)
        return f
        

In [ ]:
def train_fn(train_loader,model,optimizer,scheduler,epoch,gradient_accumulation_steps=2):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    steps=0
    total_loss=0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")
    for step, (inputs, labels) in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            with torch.cuda.amp.autocast(enabled=True):
                y_preds = model(inputs)
                loss = loss_fn(y_preds, labels)
            if gradient_accumulation_steps > 1:
                loss = loss / gradient_accumulation_steps
            
            scaler.scale(loss).backward()
            total_loss+=loss.item()
            steps+=1
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(),100)
            
            if (step+1)%2==0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

In [ ]:
def prepare_input(text,tokenizer):
    inputs=tokenizer.encode_plus(text,
                                   add_special_tokens=True,
                                   max_length=1467,
                                   padding='max_length',
                                   truncation=True)
    #inputs={k:torch.tensor(v, dtype=torch.long).unsqueeze(0) for k,v in inputs.items()}
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

    

In [ ]:
class prepare_dataset(Dataset):
    
    def __init__(self,df):
        self.text=df["full_text"].values
        self.labels=df[target_cols].values
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self,idx):
        inputs=prepare_input(self.text[idx],tokenizer)
        #for k, v in inputs.items():
            #print(f"{k} shape:", v.shape)
        labels=self.labels[idx]
        
        return inputs,labels

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,last_hidden_state,attention_mask):
        attention_mask_expanded=attention_mask.unsqueeze(-1).expand(last_hidden_state.shape).float()

        sum_embeddings=torch.sum(attention_mask_expanded*last_hidden_state,1)
        sum_mask=attention_mask_expanded.sum(1)
        sum_mask=torch.clamp(sum_mask,min=1e-9)
        mean=sum_embeddings/sum_mask
        
        return mean

class MaxPooling(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,last_hidden_state,attention_mask):
        attention_mask_expanded=attention_mask.unsqueeze(-1).expand(last_hidden_state.shape).float()
        embeddings=last_hidden_state.clone()
        embeddings[attention_mask_expanded==0]=-1e4
        max_embeddings,_=torch.max(embeddings,dim=1)
        return max_embeddings

class MinPooling(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,last_hidden_state,attention_mask):
        attention_mask_expanded=attention_mask.unsqueeze(-1).expand(last_hidden_state.shape).float()
        embeddings=last_hidden_state.clone()
        embeddings[attention_mask_expanded==0]=1e-4
        min_embeddings,_=torch.min(embeddings,dim=1)
        return min_embeddings
    
        

In [7]:
model=AutoModel.from_pretrained("ProsusAI/finbert",output_hidden_states=True)

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [8]:
model.config

BertConfig {
  "_name_or_path": "ProsusAI/finbert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [10]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [11]:
model.config

BertConfig {
  "_name_or_path": "ProsusAI/finbert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
class Model(nn.Module):
    
    def __init__(self,pool='mean',training=True):
        super().__init__()
        self.model=AutoModel.from_pretrained("microsoft/deberta-v3-base",output_hidden_states=True)
        if training:
            self.model.config.gradient_checkpointing=True
        if pool=="mean":
            self.pool=MeanPooling()
        elif pool=="max":
            self.pool=MaxPooling()
        else:
            self.pool=MeanPooling()
            
        if training:
            for param in self.model.parameters():
                param.requires_grad = False


            for param in self.model.encoder.layer[-5:].parameters():
                    param.requires_grad = True
      
        
   
    
    def extract_embeddings(self,features,inputs):
        last_hidden_state=features[0]
        #print(features.last_hidden_state.shape)
        #print(features.keys())
        #print(len(features['hidden_states']))
        #print(features['last_hidden_state']==features["hidden_states"][0])
        #print(last_hidden_state[:,512:])
        attention_mask=inputs["attention_mask"]
        embeddings=self.pool(last_hidden_state,attention_mask)
        
        return embeddings
        
    def forward(self,inputs):
        features=self.model(**inputs)
        out=self.extract_embeddings(features,inputs)
        return out
        